In [1]:
import pandas as pd

In [11]:
df = pd.read_csv('../data/분류전처리.csv')
df

,날짜,번호판,브레이크,시각,위험운전분류,차량속도분류,RPM분류,가속도분류,일일주행거리분류,방위각분류,GPS_X분류,GPS_Y분류
0,20221201,부산70자1854,0,5,0,0,0,0,0,0,129.15,35.23
1,20221201,부산70자1854,0,5,0,0,0,0,0,0,129.15,35.23
2,20221201,부산70자1854,0,5,0,0,1,0,0,0,129.15,35.23
3,20221201,부산70자1854,0,5,0,0,1,0,0,0,129.15,35.23
4,20221201,부산70자1854,0,5,0,0,0,0,0,0,129.15,35.23
...,...,...,...,...,...,...,...,...,...,...,...,...
5948207,20221231,부산70자1894,0,21,0,0,0,0,8,6,129.15,35.23
5948208,20221231,부산70자1894,0,21,0,0,0,0,8,6,129.15,35.23
5948209,20221231,부산70자1894,0,21,0,0,0,0,8,6,129.15,35.23
5948210,20221231,부산70자1894,0,21,0,0,0,0,8,6,129.15,35.23


In [3]:
import warnings
warnings.filterwarnings("ignore")
import optuna
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

from xgboost import XGBClassifier
from xgboost import plot_importance

import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
df = df.query('4000000 < index < 4010000')
df

,날짜,번호판,브레이크,시각,위험운전분류,차량속도분류,RPM분류,가속도분류,일일주행거리분류,방위각분류,GPS_X분류,GPS_Y분류
4000001,20221225,부산70자1893,1,5,0,1,2,0,0,13,129.12,35.20
4000002,20221225,부산70자1893,1,5,0,1,2,0,0,13,129.12,35.20
4000003,20221225,부산70자1893,1,5,0,1,1,0,0,13,129.12,35.20
4000004,20221225,부산70자1893,0,5,0,1,1,0,0,12,129.12,35.20
4000005,20221225,부산70자1893,0,5,0,1,1,0,0,12,129.12,35.20
...,...,...,...,...,...,...,...,...,...,...,...,...
4009995,20221225,부산70자1893,1,9,0,1,1,0,2,22,129.12,35.17
4009996,20221225,부산70자1893,1,9,0,1,1,0,2,22,129.12,35.17
4009997,20221225,부산70자1893,1,9,0,1,1,0,2,22,129.12,35.17
4009998,20221225,부산70자1893,1,9,0,1,1,0,2,22,129.12,35.17


In [19]:
df.query('위험운전분류 == 1').count()

날짜          75
번호판         75
브레이크        75
시각          75
위험운전분류      75
차량속도분류      75
RPM분류       75
가속도분류       75
일일주행거리분류    75
방위각분류       75
GPS_X분류     75
GPS_Y분류     75
dtype: int64

In [20]:
X = df[['브레이크', '시각', '차량속도분류', 'RPM분류', '가속도분류', '일일주행거리분류', '방위각분류', 'GPS_X분류', 'GPS_Y분류']]
y = df['위험운전분류']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle=True, test_size=0.3)

In [22]:
model = XGBClassifier()

In [23]:
both_scoring = {'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score), 'Loss': 'neg_log_loss'}
params = {'n_estimators': [100, 200, 500, 1000, 1500],
          'learning_rate': [0.05, 0.1, 0.2],
          'max_depth': [3, 4, 5]}

In [24]:
clf = GridSearchCV(model, params, cv=5, scoring=both_scoring, refit='AUC', return_train_score=True)
clf.fit(X_train[['브레이크', '시각', '차량속도분류', 'RPM분류', '가속도분류', '일일주행거리분류', '방위각분류', 'GPS_X분류', 'GPS_Y분류']], y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             param_grid={'learning_rate': [0.05, 0.1, 0.2],
                         'max_depth': [3, 4, 5],
                         'n_estimators': [100, 200, 500, 1000, 1500]},
             refit='AUC', return_train_score=True,
             scoring={'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score),
                      'Loss': 'neg_log_loss'})

In [25]:
print((clf.best_score_, clf.best_params_))
print('=' * 30)
print('Grid scores on training data: ')
means = clf.cv_results_['mean_test_AUC']
stds = clf.cv_results_['std_test_AUC']
log_losses = clf.cv_results_['std_test_Loss']

for mean, std, log_loss, params in zip(means, stds, log_losses, clf.cv_results_['params']):
    print(f'AUC Score: {mean} (+/-{std * 2}; Log Loss: {log_loss} for {params}')

(nan, {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100})
Grid scores on training data: 
AUC Score: nan (+/-nan; Log Loss: 0.0047311734903162555 for {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}
AUC Score: nan (+/-nan; Log Loss: 0.005181766917831923 for {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200}
AUC Score: nan (+/-nan; Log Loss: 0.005599320714119568 for {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 500}
AUC Score: nan (+/-nan; Log Loss: 0.006564524025537612 for {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 1000}
AUC Score: nan (+/-nan; Log Loss: 0.007047225904548865 for {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 1500}
AUC Score: nan (+/-nan; Log Loss: 0.00496907751220939 for {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 100}
AUC Score: nan (+/-nan; Log Loss: 0.006052732134185599 for {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 200}
AUC Score: nan (+/-nan; Log Loss: 0.006725520915033517 for 